In [1]:
from bert_codes.feature_generation import combine_features,return_dataloader,return_cnngru_dataloader
from bert_codes.data_extractor import data_collector
from bert_codes.own_bert_models import *
from bert_codes.utils import *

I0426 08:51:38.027387 139927852119872 file_utils.py:38] PyTorch version 1.1.0 available.
Using TensorFlow backend.


In [7]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
X = [[1, 2], [3, 4], [1, 2], [3, 4]]
y = [0, 0, 1, 1]
skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)
StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


TRAIN: [1 3] TEST: [0 2]


TypeError: only integer scalar arrays can be converted to a scalar index

In [8]:
def cross_validate_bert():
    total_data=pd.read_csv('Total_data_annotated.csv')
    print(total_data.head(5))

In [9]:
cross_validate_bert()

   Unnamed: 0  Mithun  Punyajoy  Soumyadeep      id  label  \
0           0       0         1           0  524726    0.0   
1           1       0         0           0  207102    0.0   
2           2       0         0           0  135627    0.0   
3           3       0         0           0  665173    0.0   
4           4       0         1           0  543553    0.0   

                                                text  \
0  आज ही के दिन 30 जनबरी को नाथूराम ने गाँधी वध क...   
1  *👉जम्मू कश्मीरः बडगाम में एनकाउंटर🔫 जारी, दो आ...   
2  *यह साध्वी जी की परिपक्वता और गहरी समझ की निशा...   
3  भारतीय सेना ने की एक और सर्जिकल स्‍ट्राइक, म्‍...   
4  एक नया ड्रामा डॉ अम्बेडकर के नाम पर प्रचलित हु...   

                                          translated  
0                                                NaN  
1  * 👉  Jammu Kashmir: Encounter continues 🔫  in ...  
2  * It is a sign of maturity and deep understand...  
3  The Indian Army carried out this operation in ...  
4  A new drama 